In [1]:
#!/usr/bin/env python
# coding: utf-8

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

hyperparameters

In [3]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

In [4]:
torch.manual_seed(1337)

wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [5]:
with open('ai.txt', 'r', encoding='utf-8') as f:
    text = f.read()

here are all the unique characters that occur in this text

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

Train and test splits

In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

data loading

In [8]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [11]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [12]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

In [13]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

super simple bigram model

In [14]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [15]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters HERE 1')

0.212051 M parameters HERE 1


create a PyTorch optimizer

In [16]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [17]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.5594, val loss 4.5570
step 100: train loss 2.7009, val loss 2.7488
step 200: train loss 2.5136, val loss 2.5503
step 300: train loss 2.4481, val loss 2.4928
step 400: train loss 2.3585, val loss 2.4362
step 500: train loss 2.2621, val loss 2.3492
step 600: train loss 2.1711, val loss 2.2804
step 700: train loss 2.0793, val loss 2.2160
step 800: train loss 2.0110, val loss 2.1947
step 900: train loss 1.9613, val loss 2.1526
step 1000: train loss 1.8909, val loss 2.1118
step 1100: train loss 1.8192, val loss 2.1055
step 1200: train loss 1.7805, val loss 2.0446
step 1300: train loss 1.7306, val loss 2.0368
step 1400: train loss 1.6865, val loss 2.0368
step 1500: train loss 1.6580, val loss 2.0371
step 1600: train loss 1.6248, val loss 1.9954
step 1700: train loss 1.5991, val loss 1.9734
step 1800: train loss 1.5739, val loss 1.9737
step 1900: train loss 1.5470, val loss 1.9633
step 2000: train loss 1.5235, val loss 1.9746
step 2100: train loss 1.4980, val loss 1.9642


generate from the model

In [41]:
print("before generating")
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(context.shape)
a = m.generate(context, max_new_tokens=500)[0]
print(a.shape)
print(a)
b = a.tolist()
print(b)
print(decode(b))
a[2] = a[2] + 5
a[5] = a[5] + 1
a[8] = a[8] + 3
a[9] = a[9] + 2
print(a)
b = a.tolist()
print(b)
print(decode(b))

before generating
torch.Size([1, 1])
torch.Size([501])
tensor([ 0,  0, 43, 72, 61, 74, 57,  1, 64, 67, 66, 59,  1, 72, 60, 57,  1, 65,
        53, 66, 77,  1, 67, 58,  1, 53, 66,  1, 80, 53, 68, 53, 53, 65, 57, 56,
         1, 25, 33,  1, 56, 61, 74, 61, 56, 57, 56,  1, 54, 77,  1, 25, 33,  6,
        66,  1, 73, 66, 55, 67, 64, 67, 70,  1, 72, 60, 53, 72,  1, 60, 53, 71,
        71, 73, 65, 57, 56,  1, 70, 53, 56, 57, 53, 55, 57, 11,  1, 44, 60, 57,
        77,  1, 74, 53, 70, 61, 57, 72, 61, 67, 66,  1, 67, 58,  1, 53, 71, 71,
        68, 70, 67, 55, 57, 71, 71,  1, 75, 61, 72, 60,  1, 53, 72, 72, 57, 65,
        68, 72,  1, 65, 57, 56, 61, 71, 61, 67, 66,  1, 55, 60, 67, 61, 66, 74,
        57, 70, 77,  8,  1, 53, 71,  1, 64, 57, 53, 70, 66, 61, 66, 59, 11, 51,
        19, 20, 52,  1, 56, 57, 56,  1, 53, 71,  1,  3, 71, 53, 58, 58, 61, 55,
        61, 53, 64,  1, 61, 66, 72, 57, 64, 64, 61, 59, 57, 66, 55, 57,  1, 53,
        59, 70, 53, 56, 11, 51, 13, 21, 20, 52,  1, 44, 75, 67, 6